In [11]:
from os import listdir
from os.path import isfile, join
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from collections import Counter, defaultdict
from urllib import request
import math

charles_dickens = ["https://www.gutenberg.org/files/98/98-0.txt", "https://www.gutenberg.org/files/46/46-0.txt", "https://www.gutenberg.org/files/1400/1400-0.txt", "https://www.gutenberg.org/files/730/730-0.txt", "https://www.gutenberg.org/files/766/766-0.txt", "https://www.gutenberg.org/cache/epub/25985/pg25985.txt", "https://www.gutenberg.org/files/676/676-0.txt", "https://www.gutenberg.org/cache/epub/1023/pg1023.txt", "https://www.gutenberg.org/cache/epub/37121/pg37121.txt", "https://www.gutenberg.org/files/42232/42232-0.txt", "https://www.gutenberg.org/cache/epub/41894/pg41894.txt", "https://www.gutenberg.org/cache/epub/1415/pg1415.txt", "https://www.gutenberg.org/cache/epub/1394/pg1394.txt"]

marry_shelly = ["https://www.gutenberg.org/files/84/84-0.txt", "https://www.gutenberg.org/files/18247/18247-0.txt", "https://www.gutenberg.org/cache/epub/15238/pg15238.txt", "https://www.gutenberg.org/cache/epub/6447/pg6447.txt", "https://www.gutenberg.org/files/56665/56665-0.txt", "https://www.gutenberg.org/files/63337/63337-0.txt", "https://www.gutenberg.org/files/63338/63338-0.txt", "https://www.gutenberg.org/files/63339/63339-0.txt", "https://www.gutenberg.org/files/64555/64555-0.txt", "https://www.gutenberg.org/files/64556/64556-0.txt", "https://www.gutenberg.org/files/64557/64557-0.txt", "https://www.gutenberg.org/cache/epub/4695/pg4695.txt"]

austin_jane = ["https://www.gutenberg.org/files/1342/1342-0.txt", "https://www.gutenberg.org/files/158/158-0.txt", "https://www.gutenberg.org/files/161/161-0.txt", "https://www.gutenberg.org/cache/epub/105/pg105.txt", "https://www.gutenberg.org/files/121/121-0.txt", "https://www.gutenberg.org/files/63569/63569-0.txt", "https://www.gutenberg.org/files/141/141-0.txt", "https://www.gutenberg.org/cache/epub/946/pg946.txt", "https://www.gutenberg.org/cache/epub/42078/pg42078.txt", "https://www.gutenberg.org/files/1212/1212-0.txt"]

mark_twain = ["https://www.gutenberg.org/files/142/142-0.txt", "https://www.gutenberg.org/files/76/76-0.txt", "https://www.gutenberg.org/files/76/76-0.txt", "https://www.gutenberg.org/files/3184/3184-0.txt", "https://www.gutenberg.org/files/3179/3179-0.txt", "https://www.gutenberg.org/cache/epub/19987/pg19987.txt",
"https://www.gutenberg.org/files/3187/3187-0.txt", "https://www.gutenberg.org/files/86/86-0.txt", "https://www.gutenberg.org/files/3192/3192-0.txt", "https://www.gutenberg.org/files/3180/3180-0.txt", "https://www.gutenberg.org/files/3178/3178-0.txt", "https://www.gutenberg.org/files/3176/3176-0.txt"]

hg_wells = ["https://www.gutenberg.org/files/59774/59774-0.txt", "https://www.gutenberg.org/files/524/524-0.txt", "https://www.gutenberg.org/cache/epub/19229/pg19229.txt", "https://www.gutenberg.org/files/59769/59769-0.txt", "https://www.gutenberg.org/files/1013/1013-0.txt", "https://www.gutenberg.org/files/456/456-0.txt", "https://www.gutenberg.org/cache/epub/11502/pg11502.txt", "https://www.gutenberg.org/cache/epub/3690/pg3690.txt", "https://www.gutenberg.org/files/1046/1046-0.txt", "https://www.gutenberg.org/files/3797/3797-0.txt", "https://www.gutenberg.org/files/5230/5230-0.txt", "https://www.gutenberg.org/cache/epub/159/pg159.txt", "https://www.gutenberg.org/cache/epub/39162/pg39162.txt", "https://www.gutenberg.org/cache/epub/11640/pg11640.txt", "https://www.gutenberg.org/files/1047/1047-0.txt", "https://www.gutenberg.org/files/60173/60173-0.txt"]


link_to_authors = [charles_dickens, marry_shelly, austin_jane, mark_twain, hg_wells]


def get_files(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

gutenberg_directory = './gutenberg'

def GetRawText(filename):
    f = open(filename, 'r', encoding="utf-8")
    raw = f.read()
    #remove non book stuff
    start_index = raw.find("***")
    end_of_line = raw.find("\n", start_index)
    return raw[end_of_line : ]

def GetUrlText(url):
    response = request.urlopen(url)
    raw = response.read().decode('utf8', "ignore")
    #remove non book stuff
    start_index = raw.find("***")
    end_of_line = raw.find("\n", start_index)
    return raw[end_of_line : ]

def get_works():
    authors = get_files(gutenberg_directory)
    for i, author in enumerate(authors):
        c = Counter([])
        filepath = gutenberg_directory + '/' + author
        text = GetRawText(filepath)
        word_list = []
        for word in word_tokenize(text):
            word = word.lower()
            if word not in stop_words and word.isalpha():
                word_list.append(word)
        c.update(word_list)
        f = open('author_{}.txt'.format(i), 'w', encoding="utf-8")
        for word, count in c.most_common():
            f.write('{} {}\n'.format(word, count))
        f.close()

# get_works()

def create_tfidf():
    link_count = 0
    collect = []
    authors = get_files(gutenberg_directory)
    for author in link_to_authors:
        word_freq = defaultdict(lambda: [])
        for i, link in enumerate(author):
            link_count += 1
            text = GetUrlText(link)
            for word in word_tokenize(text):
                word = word.lower()
                if word.isalpha() and word not in stop_words:
                    if word in word_freq:
                        freqs = word_freq[word]
                        if i not in freqs:
                            word_freq[word].append(i)
                    else:
                        word_freq[word].append(i)    
        collect.append(word_freq)
    print(link_count)
    return
    global_dict = defaultdict(lambda: 0)
    for c in collect:
        for word, doc_list in c.items():
            global_dict[word] += len(doc_list)
    f = open('tfidf.txt', 'w', encoding="utf-8")
    for word, count in global_dict.items():
        f.write('{} {}\n'.format(word, count))
    f.close()


# create_tfidf()
NUM_LINKS = 63
def calculate_idf():
    idf = []
    f = open('tfidf.txt', 'r', encoding="utf-8")
    for line in f.readlines():
        line = line.strip()
        word, count = line.split()
        count = int(count)
        idf.append((word, math.log2(NUM_LINKS / count)+ 1))
    f.close()
    f = open('tfidf_freq.txt', 'w', encoding="utf-8")
    for word, count in idf:
        f.write('{} {}\n'.format(word, count))
    f.close()

calculate_idf()



In [13]:
import nltk
from nltk.tag import pos_tag
import re

def getSentenceType(statement): 
    sentenceParts = pos_tag(statement.split()) 
    return sentenceParts

def sentSegs(taggedSent, grammar, loops):
    cp = nltk.RegexpParser(grammar, loop=loops)
    result = cp.parse(taggedSent)
    return result

grammar = r"""
        NP:
            {<.*>+}
            }<IN|CC>+{
    """

sentence = getSentenceType("He is smart, and he is punctual")
result = sentSegs(sentence, grammar, loops=1)

print(result)

[('He', 'PRP'), ('is', 'VBZ'), ('smart,', 'JJ'), ('and', 'CC'), ('he', 'PRP'), ('is', 'VBZ'), ('punctual', 'JJ')]
(S
  (NP He/PRP is/VBZ smart,/JJ)
  and/CC
  (NP he/PRP is/VBZ punctual/JJ))
